In [29]:
import os
import plotly.express as px
import pandas as pd
import numpy as np
import seaborn as sns
from datetime import timedelta
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# EXPLORATORY ANALYSIS & PREPROCESSING

In [2]:
patient_A_meals = '/una-health-data-challenge/aaaaaaaa-aaaa-aaaa-aaaa-aaaaaaaaaaaa/activities_all.csv'
patient_A_levels = '/una-health-data-challenge/aaaaaaaa-aaaa-aaaa-aaaa-aaaaaaaaaaaa/levels_all.csv'
patient_B_meals = '/una-health-data-challenge/bbbbbbbb-bbbb-bbbb-bbbb-bbbbbbbbbbbb/activities_all.csv'
patient_B_levels = '/una-health-data-challenge/bbbbbbbb-bbbb-bbbb-bbbb-bbbbbbbbbbbb/levels_all.csv'
patient_C_meals = '/una-health-data-challenge/cccccccc-cccc-cccc-cccc-cccccccccccc/activities_all.csv'
patient_C_levels = '/una-health-data-challenge/cccccccc-cccc-cccc-cccc-cccccccccccc/levels_all.csv'

In [19]:
# given te small data size, all data will be preprocessed together

# concats all subject's meal data into 1 df; selects relevant variables
data_meals = pd.concat([pd.read_csv(os.getcwd() + patient_A_meals), pd.read_csv(os.getcwd() + patient_B_meals), pd.read_csv(os.getcwd() + patient_C_meals)])
data_meals = data_meals[['user_id','record_type', 'timestamp_start']].reset_index(drop=True)

# concats all subject's glucose levels data into 1 df (reds csv file w/o header); selects relevant variables
data_levels = pd.concat([pd.read_csv(os.getcwd() + patient_A_levels, skiprows=2), pd.read_csv(os.getcwd() + patient_B_levels, skiprows=1), pd.read_csv(os.getcwd() + patient_C_levels, skiprows=2)])
data_levels = data_levels[['Seriennummer', 'Gerätezeitstempel', 'Aufzeichnungstyp', 'Glukosewert-Verlauf mg/dL', 'Glukose-Scan mg/dL']].reset_index(drop=True)

# filters out rows where Aufzeichnungstyp = 5/6
data_levels = data_levels[(data_levels.Aufzeichnungstyp != 5) | (data_levels.Aufzeichnungstyp != 6)]

# concats columns Glukosewert-Verlauf mg/dL and Glukose-Scan mg/dL into the same column; eliminates variable Aufzeichnungstyp as it is no longer needed
data_levels['Glucose Level mg/dL'] = data_levels['Glukosewert-Verlauf mg/dL'].combine_first(data_levels['Glukose-Scan mg/dL'])
data_levels.drop(['Aufzeichnungstyp', 'Glukosewert-Verlauf mg/dL', 'Glukose-Scan mg/dL'], axis=1, inplace = True)

# filters out rows where there's no glucose measurement
data_levels.dropna(subset=['Glucose Level mg/dL'], inplace=True)

# converts time stamp columns to appropriate datetime format
data_levels['Timestamp'] = pd.to_datetime(data_levels['Gerätezeitstempel'], infer_datetime_format=True)
data_levels.drop(['Gerätezeitstempel'], axis=1, inplace = True)
data_meals['timestamp_start'] = pd.to_datetime(pd.to_datetime(data_meals['timestamp_start']).dt.strftime('%Y-%m-%d %H:%M:%S'))

# sortes according to datetime, as even within patients there's mix-ups; data for inidvidual patients will be sliced from here later
data_levels = data_levels.sort_values(['Timestamp'], ascending=True)

# to avoid mix-ups, and because in these datasets each serial number correspondes to one unique user_id, serial number was replaced by user_id
data_levels['Seriennummer'].replace({"1D48A10E-DDFB-4888-8158-026F08814832":"aaaaaaaa-aaaa-aaaa-aaaa-aaaaaaaaaaaa", "006A3043-6E08-4C19-B3F1-B0D95C5737CA": "bbbbbbbb-bbbb-bbbb-bbbb-bbbbbbbbbbbb", "e09bb0f0-018b-429b-94c7-62bb306a0136": "cccccccc-cccc-cccc-cccc-cccccccccccc"}, inplace=True)
data_levels = data_levels.rename(columns={'Seriennummer':'user_id'})


In [20]:
data_meals.tail()

,user_id,record_type,timestamp_start
33,cccccccc-cccc-cccc-cccc-cccccccccccc,MEAL_DINNER,2021-02-17 19:45:00
34,cccccccc-cccc-cccc-cccc-cccccccccccc,MEAL_LUNCH,2021-02-19 11:45:00
35,cccccccc-cccc-cccc-cccc-cccccccccccc,MEAL_DINNER,2021-02-19 18:30:00
36,cccccccc-cccc-cccc-cccc-cccccccccccc,MEAL_LUNCH,2021-02-21 12:45:00
37,cccccccc-cccc-cccc-cccc-cccccccccccc,MEAL_DINNER,2021-02-21 18:00:00


In [21]:
data_meals.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38 entries, 0 to 37
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   user_id          38 non-null     object        
 1   record_type      38 non-null     object        
 2   timestamp_start  38 non-null     datetime64[ns]
dtypes: datetime64[ns](1), object(2)
memory usage: 1.0+ KB


# VISUALIZING HISTORIC BLOOD GLUCOSE LEVELS BY PATIENT

In [22]:
#gets dataframes of glucose levels for each subject individually
data_levels_A = data_levels[data_levels['user_id'] == 'aaaaaaaa-aaaa-aaaa-aaaa-aaaaaaaaaaaa']
data_levels_B = data_levels[data_levels['user_id'] == 'bbbbbbbb-bbbb-bbbb-bbbb-bbbbbbbbbbbb']
data_levels_C = data_levels[data_levels['user_id'] == 'cccccccc-cccc-cccc-cccc-cccccccccccc']

In [23]:
for patient_data, patient_id in zip([data_levels_A, data_levels_B, data_levels_C], [' A',' B',' C']):
    
    
    fig = px.line(patient_data, x='Timestamp', y='Glucose Level mg/dL', title='Historic Blood Glucose Levels for Patient{}'.format(patient_id))
    
    
# apparently one can't add annotations to a vertical line in plotly due to a reported bug; to work around it, drew a rectangle instead
#for row in data_meals_A['timestamp_start']:
    #fig.add_vrect(x0=row, x1=row, annotation_text="test", annotation_position="top left", line_color="green", opacity=0.5) 

    fig.update_xaxes(
        rangeslider_visible=True,
        rangeselector=dict(
        #buttons=list([
            #dict(count=1, label="25", step="day", stepmode="backward"),
            #dict(count=2, label="24", step="day", stepmode="backward"),
            #dict(step="all")
        #])
        )
    )
    
    fig.update_layout({'plot_bgcolor': 'rgba(.03, .03, .03, .03)'})
    
    fig.show()


In [24]:
#try adding a drop down for the different subjects
#Should I do daily average to see how it varies along the day? Ideally, 2 buttons: 'all' and 'average'.

# VISUALIZING HISTORIC BLOOD GLUCOSE LEVELS AFTER MEALS

In [33]:
#gets dataframes of meals for each subject individually
data_meals_A = data_meals[data_meals['user_id'] == 'aaaaaaaa-aaaa-aaaa-aaaa-aaaaaaaaaaaa']
data_meals_B = data_meals[data_meals['user_id'] == 'bbbbbbbb-bbbb-bbbb-bbbb-bbbbbbbbbbbb']
data_meals_C = data_meals[data_meals['user_id'] == 'cccccccc-cccc-cccc-cccc-cccccccccccc']

**4 types of meals will be considered: BREAKFAST, LUNCH, DINNER and SNACKS, the lattest including record types DRINK and MEAL_SNACK; the reason for this aggregation is to facilitate the visualization & interpretation of results, and potetially understand the effect of extra-meal snacking on the glucose-levels of the patients**


In [34]:
# explain here aggregations that you chose

meal_types = list(data_meals_C['record_type'].unique())
meal_types.append('SNACK')

for patient_data in [data_meals_A, data_meals_B, data_meals_C]:
    patient_data['record_type'] = patient_data['record_type'].replace(['MEAL_SNACK', 'DRINK'], 'SNACK')


As there are only 2 instances of physical exercise, I decided not to study the effect of physical exercise in blood glucose levels.